In [46]:
import pandas as pd

#### Nuotekos

In [70]:
nuotekos = pd.read_csv("csv/nuotekos.csv")
nuotekos = nuotekos.replace("Klaipėdos m. sav.", "Klaipėda").replace("Kauno m. sav.", "Kaunas").replace("Vilniaus m. sav.", "Vilnius").replace("Panevėžio m. sav.", "Panevėžys").replace("Šiaulių m. sav.", "Šiauliai").replace("Alytaus m. sav.", "Alytus")
nuotekos = nuotekos.drop(['Rodiklis', "Matavimo vienetai"], 1)
nuotekos = pd.merge(
    nuotekos[nuotekos["Išvalymas"] == "Išleista išvalytų iki normos nuotekų"],
    nuotekos[nuotekos["Išvalymas"] == "Iš viso išleista nuotekų"],
    on=["Administracinė teritorija", "Laikotarpis"])
nuotekos["y"] = round(100*nuotekos["Reikšmė_x"]/nuotekos["Reikšmė_y"],2)
nuotekos = nuotekos.drop(['Išvalymas_x', "Išvalymas_y", "Reikšmė_x", "Reikšmė_y"], 1)
nuotekos = nuotekos.rename({'Laikotarpis': 'x', 'Administracinė teritorija': 'city'}, axis=1)
nuotekos.to_csv("kpi/nuotekos.csv", index = False)

#### Metalo laužas

In [7]:
mlauzas = pd.read_csv("csv/metalo_lauzas.csv")
mlauzas = mlauzas[(mlauzas["Supirkimo šaltinis"] == "Supirkta, iš viso")]
mlauzas = mlauzas.drop(['Rodiklis', "Matavimo vienetai", "Supirkimo šaltinis"], 1)
mlauzas = pd.merge(
    mlauzas[mlauzas["Metalų laužo rūšis"] == "Juodųjų metalų laužas, tūkst. tonų"],
    mlauzas[mlauzas["Metalų laužo rūšis"] == "Spalvotųjų metalų laužas, tonos"],
    on=["Apskritys", "Laikotarpis"])
mlauzas["y"] = mlauzas["Reikšmė_x"] + mlauzas["Reikšmė_y"]
mlauzas = mlauzas.drop(['Metalų laužo rūšis_x', "Metalų laužo rūšis_y", "Reikšmė_x", "Reikšmė_y"], 1)
mlauzas = mlauzas.rename({'Laikotarpis': 'x', 'Apskritys': 'city'}, axis=1)
mlauzas.to_csv("kpi/mlauzas.csv", index = False)

In [175]:
cat = pd.read_csv("kpi/categories.csv")
weights = [25, 25, 25, 25]

cat.iloc[:,[2,3,4,5]] = cat.iloc[:,[2,3,4,5]] * pd.Series(weights,index = cat.columns[[2,3,4,5]]) / 25 * 100 / sum(weights)
cat["total"]= cat.iloc[:,[2,3,4,5]].sum(axis=1)

vanduo = cat[["city","vanduo","x"]]
vanduo = vanduo.rename({'vanduo': 'y'}, axis=1)
vanduo.to_csv("kpi/vanduo.csv", index = False)

transportas = cat[["city","transportas","x"]]
transportas = transportas.rename({'transportas': 'y'}, axis=1)
transportas.to_csv("kpi/transportas.csv", index = False)

oras = cat[["city","oras","x"]]
oras = oras.rename({'oras': 'y'}, axis=1)
oras.to_csv("kpi/oras.csv", index = False)

siuksles = cat[["city","šiukšles","x"]]
siuksles = siuksles.rename({'šiukšles': 'y'}, axis=1)
siuksles = siuksles[siuksles["x"] < 2020]
siuksles.to_csv("kpi/siuksles.csv", index = False)

total = cat[["city","total","x"]]
total = total.rename({'total': 'y'}, axis=1)
total.to_csv("kpi/total.csv", index = False)

#### KPI

In [189]:
total = pd.read_csv("kpi/categories.csv")
kpi = total[["x","city","total"]]
kpi = kpi.rename({'total': 'y'}, axis=1)
kpi.to_csv("kpi/kpi.csv", index = False)

#### catbarchart

In [182]:
catbar = pd.read_csv("kpi/categories.csv")
catbar["šiukšles"] = list(catbar[catbar["x"] == 2019]["šiukšles"])*6
catbar = catbar[catbar["x"] == 2020]
catbar = catbar[["city", "transportas", "vanduo", "oras", "šiukšles"]]
catbar.to_csv("kpi/lycatbar.csv", index = False)

#### Indicator scores

In [115]:
iscores = pd.read_csv("kpi/indicators_scores.csv")
weights = [25, 25, 25, 25]
trans = [2,10]
water = [3,8,9]
air = [4,5,6]
trash = [7]
categories = [trans,water,air,trash]
for i,j in zip(categories,weights):
    iscores.iloc[:,i] = iscores.iloc[:,i] * j / 25 * 100 / sum(weights)
iscores["siuksles_surinktos"] = list(iscores[iscores["x"] == 2019]["siuksles_surinktos"])*6
iscores = iscores[iscores["x"] == 2020]
cols = [col for col in iscores.columns if col not in ["city","x"]]
result = {}
for city in iscores.city:
    result[city] = [{col:list(iscores[iscores["city"] == city].loc[:,col])[0]} for col in cols]
result

{'Šiauliai': [{'kelioniu_kiekis': 7.2},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 5.1},
  {'oro_tersalai_kietosios': 0.0},
  {'oro_tersalai_no': 1.7},
  {'siuksles_surinktos': 25.0},
  {'vandens_sunaudojimas_buiciai': 0.0},
  {'vandens_sunaudojimas_energetikai': 0.0},
  {'viesojo_rida': 8.2}],
 'Kaunas': [{'kelioniu_kiekis': 7.3},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 3.5},
  {'oro_tersalai_kietosios': 0.6},
  {'oro_tersalai_no': 0.4},
  {'siuksles_surinktos': 20.4},
  {'vandens_sunaudojimas_buiciai': 0.4},
  {'vandens_sunaudojimas_energetikai': 4.5},
  {'viesojo_rida': 0.0}],
 'Panevėžys': [{'kelioniu_kiekis': 7.2},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 8.3},
  {'oro_tersalai_kietosios': 8.3},
  {'oro_tersalai_no': 3.3},
  {'siuksles_surinktos': 19.4},
  {'vandens_sunaudojimas_buiciai': 0.1},
  {'vandens_sunaudojimas_energetikai': 1.4},
  {'viesojo_rida': 7.7}],
 'Alytus': [{'kelioniu_kiekis': 12.5},
  {'nuotekos': 8.3},
  {'oro_tersalai_co': 4.5},
  {'oro_tersalai_kietosio

In [4]:

df = pd.read_csv("kpi/categories.csv")
weights = [0.2,0.6,0.3,0.3]
df.iloc[:,[2,3,4,5]] = df.iloc[:,[2,3,4,5]] * pd.Series(weights,index = df.columns[[2,3,4,5]]) / 25 * 100 / sum(weights)
df["total"]= df.iloc[:,[2,3,4,5]].sum(axis=1)

df_ranks = df.copy()
df_ranks.iloc[:,2:] = df_ranks.iloc[:,1:].groupby("x").rank(ascending=False)
df_ranks = df_ranks[df_ranks["x"] == 2020]
df_ranks = df_ranks[["city", "total"]]
df_ranks = df_ranks.rename({"total":"y"}, axis = 1)
df_ranks.to_csv("kpi/totalranks.csv")